# **Kinetic model environment**

The `kinetic model` environment allows us to simulate microbial communities and their environments using ordinary differential equations. 

To illustrate how it works, we build a simplified examplt with *E. coli* subpopulations. 

In this example, there are three subpopulations: two active and one inactive. 

- **Subpopulation A** consumes glucose and produces acetate (overflow).

- **Subpopulation B** consumes acetate and produces CO2. Both subpopulations can transition to the inactive state.


<div style="height:24px;"></div>

```css
Glucose ---> [Subpopulation A] ---> Acetate
Acetate ---> [Subpopulation B] ---> CO2

[Subpopulation A] ---> [Inactive]
[Subpopulation B] ---> [Inactive]
```

<div style="height:24px;"></div>


#### **Metabolites & Metabolome**

First, we create three `Metabolite` objects—glucose, acetate, and CO2—and group them in a `Metabolome`.


In [ ]:
#import packages for the model and visualization
from kinetic_model import Metabolite, Metabolome
from kinetic_model.visualize import GraphSpecBuilder, CytoscapeExporter
import json
#to make sure plotly renders in the notebook
import plotly.io as pio
pio.renderers.default = "notebook"
# Display Cytoscape visualization
from ipycytoscape import CytoscapeWidget
import ipywidgets as widgets



#create metabolites
glucose = Metabolite(name = "glucose", concentration = 5.0, formula ={'C': 6, 'H': 12, 'O': 6}, color = '#ff0000')
acetate = Metabolite(name = "acetate", concentration = 0.0, formula ={'C': 2, 'H': 4, 'O': 3}, color = '#003eff')
co2 = Metabolite(name = "co2", concentration = 0.0, formula ={'C': 1, 'O': 2}, color = '#00B8FF')

Inspect the objects

In [ ]:
#create metabolome
metabolome = Metabolome(metabolites = [glucose, acetate, co2])

#visualize it
metabolome.make_plot()



We can also visualize the model components in Cytoscape

In [ ]:

#cycle through a kson file
model = json.loads(metabolome.to_json(full_model=True))

# Build the graph specification
builder = GraphSpecBuilder()
graph_spec = builder.build_from_json(model)

# Export to Cytoscape
exporter = CytoscapeExporter()
cytoscape_data = exporter.export(graph_spec, layout="nice", show_edge_labels=True)

# Create the viewer
viewer = CytoscapeWidget()
viewer.graph.add_graph_from_json(cytoscape_data['elements'])
viewer.set_style(cytoscape_data['style'])
viewer.set_layout(name="preset")

# Display
display(viewer)

#### **Fedding terms**

`Feeding term` objects encode how microbial subpopulations consume metabolites and release byproducts. They are defined independently and later linked to subpopulations. 


- If the `feeding term` requires more than one input metabolite, the relation is an `AND`:


```css
[metabolite A] + [metabolite C] ---> [metabolite B]
```


- If a subpopulations has more than one feeding term, the relation is an `OR`:

```css
[metabolite A] ---> [metabolite B] + [metabolite C]

[metabolite D] ---> metabolite E]
```

- A `Boost` is an  `AND` combined with an `OR`. 

- Sequential (`XOR`) requires alternative subpopulations.


In the *E. coli* example, there are two feeding terms.

In [ ]:
#Basic usage with simple metabolome:
from kinetic_model import FeedingTerm
# Create feeding term: consumes glucose, produces lactate
feeding_term_1 = FeedingTerm(
    id="glucose_to_acetate",
    metDict={"glucose": [1.0, 1.0], "acetate": [-1.0, 0.0]}, 
    metabolome=metabolome)

feeding_term_2 = FeedingTerm(
    id="acetate_to_co2",
    metDict={"acetate": [1.0, 1.0], "co2": [-1.0, 0.0]},
    metabolome=metabolome
)


We can perform basic calculations of metabolic rates using the current metabolite concentrations defined in the `metabolome` object.

In [ ]:
# Get current concentrations
concentrations = metabolome.get_concentration()
# Calculate growth and metabolism rates
growth_contribution = feeding_term_1.intrinsicGrowth(concentrations)
metabolism_rates = feeding_term_1.intrinsicMetabolism(concentrations)

print(f"growth_contribution ft 1: {growth_contribution:.3f}")
print(f"Metabolism rates ft 1: {metabolism_rates}")


# Calculate growth and metabolism rates
growth_contribution = feeding_term_2.intrinsicGrowth(concentrations)
metabolism_rates = feeding_term_2.intrinsicMetabolism(concentrations)

print(f"growth_contribution ft 2: {growth_contribution:.3f}")
print(f"Metabolism rates ft 2: {metabolism_rates}")

Let’s visualize the glucose feeding term.

In [ ]:
model = json.loads(feeding_term_1.to_json(full_model=True))

# Build the graph specification
builder = GraphSpecBuilder()
graph_spec = builder.build_from_json(model)

# Export to Cytoscape
exporter = CytoscapeExporter()
cytoscape_data = exporter.export(graph_spec, layout="nice", show_edge_labels=True)

# Create the viewer
viewer = CytoscapeWidget()
viewer.graph.add_graph_from_json(cytoscape_data['elements'])
viewer.set_style(cytoscape_data['style'])
viewer.set_layout(name="preset")

# Display
display(viewer)

#### **Subpopulations**

Subpopulations represent microbial phenotypes—defined by their feeding terms—and their state (e.g., active or inactive)

In [ ]:
from kinetic_model import Subpopulation


# Create subpopulation
subpop_1 = Subpopulation(
    name="glucose_ecoli",
    count=1.0,
    species="E. coli",
    mumax=0.5,
    feedingTerms=[feeding_term_1],
    pHopt=7.0,
    pH_sensitivity_left=1.0,
    pH_sensitivity_right=1.0,
    Topt=37.0,
    tempSensitivity_left=1.0,
    tempSensitivity_right=1.0
    )


subpop_2 = Subpopulation(
    name="acetate_ecoli",
    count=1.0,
    species="E. coli",
    mumax=0.3,
    feedingTerms=[feeding_term_2],
    pHopt=7.0,
    pH_sensitivity_left=1.0,
    pH_sensitivity_right=1.0,
    Topt=37.0,
    tempSensitivity_left=1.0,
    tempSensitivity_right=1.0
    )

subpop_3 = Subpopulation(
    name="inactive_ecoli",
    count=1.0,
    species="E. coli",
    mumax=0.0,
    feedingTerms=[],
    state="inactive",
    pHopt=7.0,
    pH_sensitivity_left=1.0,
    pH_sensitivity_right=1.0,
    Topt=37.0,
    tempSensitivity_left=1.0,
    tempSensitivity_right=1.0
    )




# Access properties
print(f"Growth rate: {subpop_1.mumax}")
print(f"pH sensitivity at pH 7.0: {subpop_1.pHSensitivity(7.0):.3f}")
print(f"Temperature sensitivity at 37°C: {subpop_1.tempSensitivity(37.0):.3f}")

# Calculate growth and metabolism with concentration vector
growth_rate = subpop_1.intrinsicGrowth(concentrations)
metabolism_rates = subpop_1.intrinsicMetabolism(concentrations)



#### **Bacteria and Microbiomes** 

A species is represented by a `Bacteria` object, defined as a collection of subpopulations and the transition terms between them. Transitions depend on the environment and are specified as boolean or threshold functions, each combined with a rate.

For example, we can define that the glucose-consuming E. coli subpopulation transitions to the acetate-consuming subpopulation when glucose falls below $1 mM$, at a rate of $0.001.$ Conversely, when glucose rises above $1 mM$, the acetate-consuming subpopulation transitions back to glucose consumption at a rate of $0.01$.

Microbiomes are encoded as collections of species.

In [ ]:
from kinetic_model import Bacteria

# Create bacteria with conditional transitions based on resource availability

bacteria = Bacteria(
    species="E. coli",
    subpopulations={
        "glucose_consumer": subpop_1,
        "acetate_consumer": subpop_2,
        "inactive": subpop_3
        },
    connections={
        "glucose_consumer": [["acetate_consumer", "glucose <1 and acetate > 1", 0.001], ["inactive", "", 0.00001]],
        "acetate_consumer": [["glucose_consumer", "glucose > 1 and acetate < 1", 0.01], ["inactive", "", 0.0000001]],
        }
    )

# Access properties
print(f"Species: {bacteria.species}")
print(f"Subpopulations: {list(bacteria.subpopulations.keys())}")
print(f"Connections: {bacteria.connections}")
